<a href="https://colab.research.google.com/github/yuu-eguci/flower-stuff-lab/blob/main/yuueguci/cifar10_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Thu Jul  1 11:42:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Google Drive をマウントします。
# NOTE: 左のトコをポチポチやってマウントすることも出来ますが(というかそのほうがラク)
# マウントすることを明示するほうが好みなのでしています。
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Keras のバージョンが結構重要な感あります。確認しておきます。
!pip list | grep -e Keras -e tensorflow -e h5py -e Pillow -e opencv-python

h5py                          3.1.0              
Keras                         2.4.3              
Keras-Preprocessing           1.1.2              
opencv-python                 4.1.2.30           
Pillow                        7.1.2              
tensorflow                    2.5.0              
tensorflow-datasets           4.0.1              
tensorflow-estimator          2.5.0              
tensorflow-gcs-config         2.5.0              
tensorflow-hub                0.12.0             
tensorflow-metadata           1.0.0              
tensorflow-probability        0.12.1             


In [7]:
# NOTE: Colaboratory で module imported but unused とか出す方法あるのかな?
# NOTE: この内容なら、 Colab では pip install 不要です。
from pathlib import Path
import numpy
from PIL import Image
# 「Oreilly requrements.txt に従う」「from keras.modules」の条件下ではエラーが出ます。
# NOTE: AttributeError:
#       module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'
from tensorflow.keras.models import load_model

MODEL_PATH = '/content/drive/MyDrive/hdf5/cifar10_fitting.hdf5'
IMAGES_FOLDER = '/content/drive/MyDrive/images-for-prediction/for-cifar10'
CIFAR10_CLASSES = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck',
]

# load model
model = load_model(MODEL_PATH)
IMAGE_SHAPE = (32, 32, 3)


# load images
def crop_resize(image_path):
    image = Image.open(image_path)
    length = min(image.size)
    crop = image.crop((0, 0, length, length))
    resized = crop.resize(IMAGE_SHAPE[:2])  # use width x height
    img = numpy.array(resized).astype('float32')
    img /= 255
    return img


folder = Path(IMAGES_FOLDER)
image_paths = [str(f) for f in folder.glob('*.png')]
images = [crop_resize(p) for p in image_paths]
images = numpy.asarray(images)

# model.predict_classes(images) は deprecated です。
# UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01.
# Please use instead:* `np.argmax(model.predict(x), axis=-1)`, if your model does multi-class classification
# というわけで numpy.argmax(model.predict(x), axis=-1) 使います。
predicted = numpy.argmax(model.predict(images), axis=-1)
# predicted は予測された class の index です。数値じゃわからないので、 CIFAR10_CLASSES を使って名称に変換しています。
print([CIFAR10_CLASSES[i] for i in predicted])

['cat']
